In [1]:
import os

os.environ["CHECKPOINTS_PATH"] = "../checkpoints"

import json
import shutil
import traceback
from concurrent.futures import ThreadPoolExecutor
from pathlib import Path

import cv2
import numpy as np
import torch
import torchvision.transforms.functional as F
from sqlalchemy.orm import Session
from src.api.db import engine
from src.api.models.db import Recording
from src.api.services import gaze_service
from src.config import CHECKPOINTS_PATH, GAZE_FOV, TOBII_FOV_X, TOBII_GLASSES_FPS, TOBII_GLASSES_RESOLUTION
from torchvision.ops import masks_to_boxes
from torchvision.transforms import InterpolationMode
from tqdm import tqdm
from ultralytics import FastSAM
from src.api.services import simrooms_service

from experiment.settings import (
    FULLY_LABELED_RECORDINGS,
    GAZE_SEGMENTATION_RESULTS_PATH,
)
import tempfile


In [2]:
with Session(engine) as session:
    trial_recordings = (
        session.query(Recording).filter(Recording.id.in_(FULLY_LABELED_RECORDINGS)).all()
    )

# Segmenting and Tracking based on Gaze Data, and grounding based on previously built Vector Index

There's a few considerations that might be interesting in an experimental context:
1. Selection of `k` in top-k results from the database?
2. Segmentation quality (IOU?, Confidence?)
3. Adding padding to the bounding boxes?
4. Indexing, search parameters? (which ones exist)
5. Merging of same-frame ROIs or not?
6. Importance of metrics (average, min, max, variance, ?)

In [3]:
class GazeSegmentationJob:
    def __init__(
        self,
        recording_id: int,
        results_path: Path,
        fovea_fov: float = GAZE_FOV,
        fov_x: float = TOBII_FOV_X,
        checkpoint_path: str = "checkpoints/FastSAM-x.pt",
        frames_path: Path | None = None,
        filter_by_gaze: bool = True
    ):
        self.fovea_fov = fovea_fov
        self.fov_x = fov_x
        self.filter_by_gaze = filter_by_gaze

        # Set up the results directory.
        self.results_path = results_path
        if self.results_path.exists():
            shutil.rmtree(self.results_path, ignore_errors=True)
            self.results_path.mkdir(parents=True, exist_ok=True)
        self.results_path.mkdir(parents=True, exist_ok=True)

        # Extract frames to a temporary directory.
        if frames_path is None:
            frames, tmp_frames_dir = simrooms_service.extract_tmp_frames(recording_id)
            self.tmp_frames_dir = tmp_frames_dir # save this so the temp dir doesn't get garbage collected
            self.frames_path = Path(tmp_frames_dir.name)
        else:
            self.frames_path = frames_path
            frames = list(self.frames_path.iterdir())
            if not self.frames_path.exists():
                raise FileNotFoundError(f"Frames path {self.frames_path} does not exist.")

        # Load the FastSAM model.
        self.model = FastSAM(checkpoint_path)

        # Video properties.
        self.resolution = TOBII_GLASSES_RESOLUTION
        self.aspect_ratio = self.resolution[1] / self.resolution[0]  # W / H
        self.fps = TOBII_GLASSES_FPS
        self.viewed_radius = int((self.fovea_fov / self.fov_x) * self.resolution[1])
        self.frame_count = len(frames)

        # Map frame indexes to gaze points.
        self.frame_to_gaze_position = gaze_service.get_gaze_position_per_frame(
            recording_id=recording_id,
            frame_count=self.frame_count,
            resolution=self.resolution,
            fps=self.fps,
        )

    def mask_too_large(self, mask: torch.Tensor) -> bool:
        """
        Check if the mask area is less than or equal to 30% of the frame area.

        Args:
            mask: A tensor containing a single mask of shape (H, W)

        Returns:
            bool: True if the mask's area is less than or equal to 30% of the frame area, False otherwise.
        """
        MAX_MASK_AREA = 0.1
        height, width = mask.shape
        frame_area = height * width
        max_mask_area = MAX_MASK_AREA * frame_area

        mask_area = mask.sum()
        return mask_area >= max_mask_area

    def run(self):
        frame_paths = list(self.frames_path.iterdir())
        frame_paths.sort(key=lambda x: int(x.stem))

        with ThreadPoolExecutor() as executor:
            for frame_path in frame_paths:
                frame_idx = int(frame_path.stem)
                results = self.model.track(
                    source=str(frame_path), imgsz=640, verbose=False, persist=True
                )[0]

                try:
                    gaze_position = self.frame_to_gaze_position.get(frame_idx)
                    if gaze_position is None:
                        continue

                    boxes = []
                    rois = []
                    masks = []
                    object_ids = []
                    confidences = []
                    for i in range(len(results.boxes)):
                        mask = F.resize(
                            results.masks[i].data,
                            self.resolution,
                            interpolation=InterpolationMode.NEAREST,
                        ).squeeze()

                        if not self.mask_too_large(mask) and (gaze_service.mask_was_viewed(mask, gaze_position) or not self.filter_by_gaze) and mask.any():
                            box = masks_to_boxes(mask.unsqueeze(0)).int().cpu().numpy()[0]
                            x1, y1, x2, y2 = box
                            roi = results.orig_img[y1:y2, x1:x2, :]
                            roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
                            boxes.append(box)
                            masks.append(mask.cpu().numpy().astype(np.uint8))
                            rois.append(roi)
                            object_ids.append(int(results.boxes.id[i]))
                            confidences.append(float(results.boxes[i].conf))


                    if len(boxes) > 0:
                        rois_array = np.empty(len(rois), dtype=object)
                        for i, roi in enumerate(rois):
                            rois_array[i] = roi

                        masks_array = np.empty(len(masks), dtype=object)
                        for i, mask in enumerate(masks):
                            masks_array[i] = mask

                        # Offload saving with thread pool (asynchronously)
                        executor.submit(
                            np.savez_compressed,
                            self.results_path / f"{frame_idx}.npz",
                            boxes=boxes,
                            rois=rois_array,
                            masks=masks_array,
                            object_ids=object_ids,
                            frame_idx=frame_idx,
                            gaze_position=gaze_position,
                            confidences=confidences,
                        )

                except Exception as e:
                    print(f"Error processing frame {frame_idx}: {e}")
                    traceback.print_exc()

In [ ]:
FILTER_RESULTS = True

if FILTER_RESULTS:
    if GAZE_SEGMENTATION_RESULTS_PATH.exists():
        shutil.rmtree(GAZE_SEGMENTATION_RESULTS_PATH, ignore_errors=True)
    GAZE_SEGMENTATION_RESULTS_PATH.mkdir(parents=True, exist_ok=True)
else:
    UNFILTERED_GAZE_SEGMENTATION_RESULTS_PATH = Path("data/unfiltered_gaze_segmentation_results")
    if UNFILTERED_GAZE_SEGMENTATION_RESULTS_PATH.exists():
        shutil.rmtree(UNFILTERED_GAZE_SEGMENTATION_RESULTS_PATH, ignore_errors=True)
    UNFILTERED_GAZE_SEGMENTATION_RESULTS_PATH.mkdir(parents=True, exist_ok=True)

FRAMES_PATHS = Path("data/recording_frames")

def process_recording(recording: Recording):
    """
    Process a recording for gaze segmentation.
    """
    recording_id = recording.id
    frames_path = FRAMES_PATHS / recording_id

    if FILTER_RESULTS:
        results_path = GAZE_SEGMENTATION_RESULTS_PATH / recording_id
    else:
        results_path = UNFILTERED_GAZE_SEGMENTATION_RESULTS_PATH / recording_id

    if results_path.exists():
        shutil.rmtree(results_path, ignore_errors=True)
    results_path.mkdir(parents=True, exist_ok=True)

    job = GazeSegmentationJob(
        recording_id=recording_id,
        results_path=results_path,
        fovea_fov=GAZE_FOV,
        fov_x=TOBII_FOV_X,
        checkpoint_path=CHECKPOINTS_PATH / "FastSAM-x.pt",
        frames_path=frames_path,
        filter_by_gaze=FILTER_RESULTS,
    )
    job.run()

for recording in tqdm(trial_recordings, desc="Processing recordings"):
    if recording.id in FULLY_LABELED_RECORDINGS:
        process_recording(recording)

Processing recordings:   0%|          | 0/10 [00:00<?, ?it/s]

Processing recordings:  10%|█         | 1/10 [02:15<20:22, 135.82s/it]

Processing recordings:  20%|██        | 2/10 [03:43<14:20, 107.53s/it]

Processing recordings:  70%|███████   | 7/10 [11:15<04:21, 87.04s/it] 

Processing recordings:  90%|█████████ | 9/10 [14:19<01:28, 88.99s/it]

Processing recordings: 100%|██████████| 10/10 [15:51<00:00, 95.19s/it]


: 

# Rendering the results

In [ ]:
from src.api.utils import image_utils
import matplotlib.pyplot as plt
import tempfile
from src.utils import extract_frames_to_dir

In [ ]:
for recording_id in FULLY_LABELED_RECORDINGS:
    SEG_RESULTS_PATH = UNFILTERED_GAZE_SEGMENTATION_RESULTS_PATH / recording_id
    VIDEO_PATH = Path("data/recordings") / f"{recording_id}.mp4"

    temp_video_frames_path = Path(tempfile.gettempdir()) / f"{recording_id}"
    if temp_video_frames_path.exists():
        shutil.rmtree(temp_video_frames_path, ignore_errors=True)
    temp_video_frames_path.mkdir(parents=True, exist_ok=True)

    extract_frames_to_dir(
        video_path=VIDEO_PATH,
        frames_path=temp_video_frames_path,
    )

    frames = list(temp_video_frames_path.iterdir())
    seg_results = list(SEG_RESULTS_PATH.iterdir())
    seg_results.sort(key=lambda x: int(x.stem))

    frame_id_to_path = {int(frame.stem): frame for frame in frames}

    for i, results in enumerate(tqdm(seg_results)):
        frame_idx = int(results.stem)
        frame_path = frame_id_to_path[frame_idx]

        frame = cv2.imread(str(frame_path))
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        results_file = np.load(results, allow_pickle=True)

        boxes = results_file["boxes"]
        masks = results_file["masks"]
        object_ids = results_file["object_ids"]

        combined_mask = np.zeros((frame.shape[0], frame.shape[1]), dtype=np.uint8)
        for mask in masks:
            combined_mask = np.logical_or(combined_mask, mask)

        image_utils.draw_mask(
            img=frame,
            mask=combined_mask,
            box=(0, 0, frame.shape[1], frame.shape[0]),
        )

        for j in range(len(boxes)):
            image_utils.draw_labeled_box(
                img=frame, box=tuple(boxes[j]), label=f"ID: {object_ids[j]}", color="#FF0000"
            )

        # save back to original path
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        cv2.imwrite(str(frame_path), frame)

    import subprocess

    cmd = f'ffmpeg -hwaccel cuda -y -pattern_type glob -framerate {TOBII_GLASSES_FPS} -i "{temp_video_frames_path!s}/*.jpg" -c:v libx264 -pix_fmt yuv420p "test.mp4"'
    subprocess.run(cmd, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

 58%|█████▊    | 872/1498 [04:21<03:07,  3.33it/s]


KeyboardInterrupt: 

: 